# Environment

In [ ]:
import numpy as np
import json
from datetime import datetime
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# Data Loading

In [ ]:
data_path = "../scrape/data/data_sentiment.npy"
dates_path = "../scrape/data/dates.json"
tweets_path = "../scrape/data/tweets.json"

data = np.load(data_path)

with open(dates_path) as json_file:  
    dates = json.load(json_file)
    
with open(tweets_path) as json_file:  
    tweets = json.load(json_file)

# Regression

In [ ]:
def linear_regression_model(data):
    model = LinearRegression().fit(data[:,0:data.shape[1]-1], data[:,data.shape[1]-1])
    return model

# Estimation

In [ ]:
def estimation_backdoor(model, data, do_x):
    num = 0
    total = 0
    
    for i in range(len(data)):
        if data[i, data.shape[1]-2] == do_x:
            num = num + 1
            total_add = model.predict(data[i, 0:data.shape[1]-1].reshape(1,-1))[0]
            total = total + total_add
    
    total = total/num
    
    return total

In [ ]:
def average_treatment_estimation_backdoor(model, data, do_x_1, do_x_2):
    num_1 = 0
    total_1 = 0
    
    num_2 = 0
    total_2 = 0
    
    for i in range(len(data)):
        if data[i, data.shape[1]-2] == do_x_1:
            num_1 = num_1 + 1
            total_add = model.predict(data[i, 0:data.shape[1]-1].reshape(1,-1))[0]
            total_1 = total_1 + total_add
            
        if data[i, data.shape[1]-2] == do_x_2:
            num_2 = num_2 + 1
            total_add = model.predict(data[i, 0:data.shape[1]-1].reshape(1,-1))[0]
            total_2 = total_2 + total_add
    
    total_1 = total_1/num_1
    total_2 = total_2/num_2
    
    return total_2 - total_1

In [ ]:
do_x_1 = 0
do_x_2 = 2
model = linear_regression_model(data)
ATE_backdoor = average_treaatment_estimation_backdoor(model, data, do_x_1, do_x_2)

# Analysis

In [ ]:
do_estimates = []
model = linear_regression_model(data)

for i in range(3):
    do_estimates.append(estimation_backdoor(model, data, i))

In [ ]:
plt.plot(range(10), do_estimates)
plt.ylabel('Expected Stock Price')
plt.xlabel('Number of Tweets')
plt.title(r'E[Y|do(X)]')
plt.show()